In [7]:
from osgeo import gdal, osr
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import os
import numpy as np
import pyproj
from shapely.wkt import loads
from shapely.geometry import Point
from shapely.ops import unary_union
from shapely import speedups
from matplotlib.ticker import MultipleLocator
from scipy.interpolate import UnivariateSpline
import warnings
from scipy.signal import find_peaks
warnings.filterwarnings("ignore")

## 1. Extract values to points for training dataset construction.
- step 1: we got 10 paired NEON upscaled traits and PRISMA reflectance. the dates coincident within 0~12 days.
- step 2: transfer the georeferenced NEON upscaled trait maps to points (in this case we use the vegetation fraction).
- step 3: select the points that the vegetation fraction greater than 0.6
- step 4: extract the land cover types to points.
- step 5: extract 5 traits values to points and excluded those values lower than 0, as well as nan values.
- step 6: extract PRISMA reflectance to points and exclude clouds and clouds shaded area.

In [2]:
def raster_to_points(geotiff, shp_name):
    inDs = gdal.Open(geotiff)
    DsoutDs = gdal.Translate(f"{shp_name}.xyz", inDs, format='XYZ', creationOptions=["ADD_HEADER_LINE=YES"])
    outDs = None
    try:
        os.remove(f'{shp_name}.csv')
    except OSError:
        pass

    os.rename(f'{shp_name}.xyz', f'{shp_name}.csv')
    os.system('ogr2ogr -f "ESRI Shapefile" -oo X_POSSIBLE_NAMES=X* -oo Y_POSSIBLE_NAMES=Y* -oo KEEP_GEOM_COLUMNS=NO {0}.shp {0}.csv'.format(shp_name))
    try:
        os.remove(f'{shp_name}.csv')
    except OSError:
        pass
    crs_wkt = inDs.GetProjection()
    shp_layer = gpd.read_file(f"{shp_name}.shp")
    shp_layer.crs = crs_wkt
    shp_layer.to_file(f"{shp_name}.shp")
    return

land_cover_type = {10: "Rainfed cropland",11: "Herbaceous cover cropland",12: "Tree or shrub cover (Orchard) cropland",
                   20: "Irrigated cropland",51: "Open evergreen broadleaved forest",52: "Closed evergreen broadleaved forest",
                   61: "Open deciduous broadleaved forest",62: "Closed deciduous broadleaved forest",71: "Open evergreen needle-leaved forest",
                   72: "Closed evergreen needle-leaved forest",81: "Open deciduous needle-leaved forest",82: "Closed deciduous needle-leaved forest",
                   91: "Open mixed leaf forest (broadleaved and needle-leaved)",92: "Closed mixed leaf forest (broadleaved and needle-leaved)", 
                   120: "Shrubland",121: "Evergreen shrubland",122: "Deciduous shrubland",130: "Grassland",140: "Lichens and mosses",
                   150: "Sparse vegetation",152: "Sparse shrubland",153: "Sparse herbaceous",181: "Swamp",182: "Marsh",183: "Flooded flat",
                   184: "Saline",185: "Mangrove",186: "Salt marsh",187: "Tidal flat",190: "Impervious surfaces",200: "Bare areas",
                   201: "Consolidated bare areas",202: "Unconsolidated bare areas",210: "Water body",220: "Permanent ice and snow",
                   0: "Filled value",250: "Filled value"}

In [16]:
flightname = ["NEON_2020_D10_CPER_20200913","NEON_2021_D10_CPER_20210525","NEON_2021_D10_CPER_20210608","NEON_2020_D13_MOAB_20200705",
              "NEON_2021_D13_MOAB_20210429","NEON_2021_D03_OSBS_20210924","NEON_2021_D07_MLBS_20210617","NEON_2021_D14_JORN_20210826",
              "NEON_2021_D14_JORN_20210909","NEON_2021_D16_WREF_20210724"]

## data path
neon_data_path = "/Volumes/UW_Madison/0_PhD_dissertation_data/1_NEON_AOP_trait_maps/4_upscaled_data/"
prisma_data_path = "/Volumes/UW_Madison/0_PhD_dissertation_data/2_PRISMA_L2D/2_PRISMA_L2D_tif_2020_2023/"
landuse_path = "/Volumes/UW_Madison/0_PhD_dissertation_data/3_GLC_FCS30D_Land_cover_data_US/4_land_cover_NEON_sites/"
out_path = "/Volumes/UW_Madison/0_PhD_dissertation_data/4_Extract_training_data/1_original_extraction/"

## upscaled NEON trait maps folder
neon_imagery = [f"{neon_data_path}{x}" for x in flightname]

## PRISMA reflectance
prisma_imagery = [f"{prisma_data_path}D10_CPER/PRS_L2D_STD_20200914175311_20200914175315_0001_HCO_FULL",
                  f"{prisma_data_path}D10_CPER/PRS_L2D_STD_20210527174644_20210527174649_0001_HCO_FULL",
                  f"{prisma_data_path}D10_CPER/PRS_L2D_STD_20210614175648_20210614175652_0001_HCO_FULL",
                  f"{prisma_data_path}D13_MOAB/PRS_L2D_STD_20200702181741_20200702181745_0001_HCO_FULL",
                  f"{prisma_data_path}D13_MOAB/PRS_L2D_STD_20210511181051_20210511181056_0001_HCO_FULL",
                  f"{prisma_data_path}D03_OSBS/PRS_L2D_STD_20210926161646_20210926161650_0001_HCO_FULL",
                  f"{prisma_data_path}D07_MLBS/PRS_L2D_STD_20210608161711_20210608161716_0001_HCO_FULL",
                  f"{prisma_data_path}D14_JORN/PRS_L2D_STD_20210817180252_20210817180257_0001_HCO_FULL",
                  f"{prisma_data_path}D14_JORN/PRS_L2D_STD_20210909175942_20210909175946_0001_HCO_FULL",
                  f"{prisma_data_path}D16_WREF/PRS_L2D_STD_20210729190927_20210729190932_0001_HCO_FULL"]

## output shapefiles stored extracted values of paired trait and reflectance
shp_path = [f"{out_path}{x}_extracted_points" for x in flightname]

In [ ]:
for i in range(len(flightname)):
    neon = neon_imagery[i]
    flight = flightname[i]
    prisma = prisma_imagery[i]
    shp_name = shp_path[i]
    year, site= flight.split("_")[1], "_".join(flight.split("_")[2:4])
    
    ## raster to points shapefile
    geotiff = f"{neon}/{flight}_vegetation_fraction_modified.tif"
    raster_to_points(geotiff, shp_name)
    
    points = gpd.read_file(f"{shp_name}.shp")
    points["vege_frac"] = points["Z"].astype(float)
    points = points[points["vege_frac"]>0.5]
    points.drop(columns=['Z'],inplace = True)
    points.reset_index(drop = True, inplace = True)
    
    ## extract land use values to points
    landuse_data = f"{landuse_path}{site}/{site}_{year}_land_cover.tif"
    land_ds = gdal.Open(landuse_data)
    
    band = land_ds.GetRasterBand(1)
    extracted_values = []
    for index, row in points.iterrows():
        point = row.geometry
        x, y = point.x, point.y
        # Convert point coordinates to pixel coordinates
        px = int((x - land_ds.GetGeoTransform()[0]) / land_ds.GetGeoTransform()[1])
        py = int((y - land_ds.GetGeoTransform()[3]) / land_ds.GetGeoTransform()[5])
        # Read value from GeoTIFF
        value = band.ReadAsArray(px, py, 1, 1)[0][0]
        extracted_values.append(value)

    extracted_values = [land_cover_type[x] for x in extracted_values]
    points["LULC"] = extracted_values

    # extract trait values to points
    tr_name = ["Chlorophylls_area","Carotenoids_area","LMA","EWT","Nitrogen"]
    col_name = {"Chlorophylls_area":"Chla+b","Carotenoids_area":"Ccar","LMA":"LMA","EWT":"EWT","Nitrogen":"Nitrogen"}
    for tr in tr_name:
        in_tif = f"{neon}/{flight}_{tr}_clipped_aggregated_modified.tif"
        tiff_ds = gdal.Open(in_tif)
        
        band = tiff_ds.GetRasterBand(1)
        extracted_values = []
        for index, row in points.iterrows():
            point = row.geometry
            x, y = point.x, point.y
            # Convert point coordinates to pixel coordinates
            px = int((x - tiff_ds.GetGeoTransform()[0]) / tiff_ds.GetGeoTransform()[1])
            py = int((y - tiff_ds.GetGeoTransform()[3]) / tiff_ds.GetGeoTransform()[5])
            # Read value from GeoTIFF
            value = band.ReadAsArray(px, py, 1, 1)[0][0]
            extracted_values.append(value)
        points[col_name[tr]] = extracted_values
    
    points = points[(points["Chla+b"]>0)&(points["Ccar"]>0)&(points["EWT"]>0)&(points["LMA"]>0)&(points["Nitrogen"]>0)]
    points.dropna(inplace = True)
    points.reset_index(drop = True, inplace = True)
    
    ## extract PRISMA reflectance values to points
    in_tif = f"{prisma}.tif"
    in_wl = f"{prisma}.wvl"
    
    tiff_ds = gdal.Open(in_tif)
    # Get number of bands in GeoTIFF
    num_bands = tiff_ds.RasterCount
    
    # Extract values for each point for each band
    extracted_values = [[] for _ in range(num_bands)]
    for index, row in points.iterrows():
        point = row.geometry
        x, y = point.x, point.y
        
        # Convert point coordinates to pixel coordinates
        px = int((x - tiff_ds.GetGeoTransform()[0]) / tiff_ds.GetGeoTransform()[1])
        py = int((y - tiff_ds.GetGeoTransform()[3]) / tiff_ds.GetGeoTransform()[5])
        
        # Read values from GeoTIFF for each band
        for band_num in range(1, num_bands + 1):
            band = tiff_ds.GetRasterBand(band_num)
            value = band.ReadAsArray(px, py, 1, 1)[0][0]
            extracted_values[band_num - 1].append(value)

    df = pd.read_csv(in_wl,delimiter=" ")
    df['wl'] = round(df['wl'],2)
    df['wl'] = df['wl'].astype(str)
    wl = list(df['wl'])
    extracted_values = pd.DataFrame(np.array(extracted_values)).T
    extracted_values.columns = wl
    points = pd.concat([points, extracted_values], axis = 1)
    
    points = points[points.iloc[:,8:28].mean(axis = 1)<0.15] ## exclude clouds (blue bands lower than 0.15)
    points = points[points.iloc[:,63:83].mean(axis = 1)>0.2] ## exclude clouds shaded area (NIR bands greater than 0.2)

    points.dropna(inplace = True)
    points.reset_index(drop = True, inplace = True)
    points.to_file(f"{shp_name}.shp")
    print(i+1,"Finished", os.path.basename(neon))

## 2. Polish the extracted training data (QGIS).
- look into details in QGIS softeware.
- vegetation fraction greater than 0.6.
- exclude the clouds and the shaded area.
- exclude the points in the edge of NEON AOP flight lines.
- exclude the points in the horizonal flight lines.
- exclude the points in the land cover data that exhibited non-vegetation.
- exclude neighboring points within a 100-meter radius to mitigate spatial autocorrelation.

In [54]:
def filter_points(gdf, min_distance):
    gdf_copy = gdf.copy()
    to_remove = []

    for index, row in gdf_copy.iterrows():
        if index not in to_remove:
            distances = gdf_copy.geometry.distance(row.geometry)
            close_points = distances[distances < min_distance].index.tolist()
            close_points.remove(index)
            to_remove.extend(close_points)

    gdf_copy.drop(index=to_remove, inplace=True)
    gdf_copy.reset_index(drop = True, inplace = True)
    return gdf_copy

data_path = "/Volumes/UW_Madison/0_PhD_dissertation_data/4_Extract_training_data/1_original_extraction/"
out_path = "/Volumes/UW_Madison/0_PhD_dissertation_data/4_Extract_training_data/2_processing_extraction/"

file_name = os.listdir(data_path)
file_name = [x for x in file_name if ".shp" in x and "._" not in x]

for file in file_name:
    print(file)
    points = gpd.read_file(f'{data_path}{file}')
    filtered_points = filter_points(points, 100)
    filtered_points.to_file(f'{out_path}{file}')

## 3. Merge the polished data to one file.
- add the necessary arttributes like the date, sites in the polished data.
- merge the extracted points data of each sites to one file (*.csv format).
- follow the Zhihui (https://doi.org/10.1111/nph.16711), Giulia (https://doi.org/10.1016/j.isprsjprs.2022.03.014) and Jochem's (https://doi.org/10.1016/j.isprsjprs.2021.06.017) paper to see how to prosess the extracted reflectance (smooth, vector normalization, exclude the water absorption bands, etc.).

In [ ]:
## Add necessary arttributes to each shapfile and save as *.csv files

shp_path = "/Volumes/UW_Madison/0_PhD_dissertation_data/4_Extract_training_data/2_processing_extraction/"
out_path = "/Volumes/UW_Madison/0_PhD_dissertation_data/4_Extract_training_data/3_save_to_csv/"

PRISMA_data = {'NEON_2020_D10_CPER_20200913_extracted_points.shp':"20200914",
               'NEON_2020_D13_MOAB_20200705_extracted_points.shp':"20200702",
               'NEON_2021_D03_OSBS_20210924_extracted_points.shp':"20210926",
               'NEON_2021_D07_MLBS_20210617_extracted_points.shp':"20210608",
               'NEON_2021_D13_MOAB_20210429_extracted_points.shp':"20210511",
               'NEON_2021_D10_CPER_20210525_extracted_points.shp':"20210527",
               'NEON_2021_D10_CPER_20210608_extracted_points.shp':"20210614",
               'NEON_2021_D14_JORN_20210826_extracted_points.shp':"20210817",
               'NEON_2021_D14_JORN_20210909_extracted_points.shp':"20210909",
               'NEON_2021_D16_WREF_20210724_extracted_points.shp':"20210729"}

file_name = os.listdir(shp_path)
file_name = [x for x in file_name if ".shp" in x and "._" not in x]
for file in file_name:
    print(file)
    data = gpd.read_file(f"{shp_path}{file}")
    data.insert(7,"site",file.split("_")[3],allow_duplicates=False)
    data.insert(8, "NEON_date", file.split("_")[4], allow_duplicates=False)
    data.insert(9, "HYP_date", PRISMA_data[file], allow_duplicates=False)
    data.insert(10, "X", data['geometry'].x, allow_duplicates=False)
    data.insert(11, "Y", data['geometry'].y, allow_duplicates=False)
    data.insert(12, "crs", data.crs.name, allow_duplicates=False)
    data.drop(columns = ["geometry"],inplace = True)
    data.to_csv(f"{out_path}{file[:-4]}.csv",index = False)

## 4. Add the NBAR reflectance and LAI into training data.

In [ ]:
pair_neon_prisma = {"NEON_2020_D10_CPER_20200913_extracted_points.shp":"PRS_L2D_STD_20200914175311_20200914175315_0001_HCO_FULL",
                    "NEON_2021_D10_CPER_20210525_extracted_points.shp":"PRS_L2D_STD_20210527174644_20210527174649_0001_HCO_FULL",
                    "NEON_2021_D10_CPER_20210608_extracted_points.shp":"PRS_L2D_STD_20210614175648_20210614175652_0001_HCO_FULL",
                    "NEON_2020_D13_MOAB_20200705_extracted_points.shp":"PRS_L2D_STD_20200702181741_20200702181745_0001_HCO_FULL",
                    "NEON_2021_D13_MOAB_20210429_extracted_points.shp":"PRS_L2D_STD_20210511181051_20210511181056_0001_HCO_FULL",
                    "NEON_2021_D03_OSBS_20210924_extracted_points.shp":"PRS_L2D_STD_20210926161646_20210926161650_0001_HCO_FULL",
                    "NEON_2021_D07_MLBS_20210617_extracted_points.shp":"PRS_L2D_STD_20210608161711_20210608161716_0001_HCO_FULL",
                    "NEON_2021_D14_JORN_20210826_extracted_points.shp":"PRS_L2D_STD_20210817180252_20210817180257_0001_HCO_FULL",
                    "NEON_2021_D14_JORN_20210909_extracted_points.shp":"PRS_L2D_STD_20210909175942_20210909175946_0001_HCO_FULL",
                    "NEON_2021_D16_WREF_20210724_extracted_points.shp":"PRS_L2D_STD_20210729190927_20210729190932_0001_HCO_FULL"}

data_path = "/Volumes/ChenLab/Fujiang/0_Seasonal_PRISMA_traits/4_Extract_training_data/2_processing_extraction/NBAR_refl/"
neon_data_path = "/Volumes/ChenLab/Fujiang/0_Seasonal_PRISMA_traits/1_NEON_AOP_trait_maps/4_upscaled_data/"
prisma_data_path = "/Volumes/ChenLab/Fujiang/0_Seasonal_PRISMA_traits/2_PRISMA_L2D/3_PRISMA_full_band_data/6_BRDF_correction/"
wvl_path = "/Volumes/ChenLab/Fujiang/0_Seasonal_PRISMA_traits/2_PRISMA_L2D/4_PRISMA_wvl_data/"

file_name = os.listdir(data_path)
file_name = [x for x in file_name if ".shp" in x and "._" not in x]

for file in file_name:
    points = gpd.read_file(f'{data_path}{file}')
    
    site = ("_").join(file.split("_")[2:4])
    imagery_path = f"{prisma_data_path}{site}"
    wvl_folder = f"{wvl_path}{site}"
    prisma = pair_neon_prisma[file]

    in_tif = f"{imagery_path}/{prisma}_NBAR.tif"
    in_wl = f"{wvl_folder}/{prisma}.wvl"
    tiff_ds = gdal.Open(in_tif)
   
    num_bands = tiff_ds.RasterCount
    
    extracted_values = [[] for _ in range(num_bands)]
    for index, row in points.iterrows():
        point = row.geometry
        x, y = point.x, point.y
        
        # Convert point coordinates to pixel coordinates
        px = int((x - tiff_ds.GetGeoTransform()[0]) / tiff_ds.GetGeoTransform()[1])
        py = int((y - tiff_ds.GetGeoTransform()[3]) / tiff_ds.GetGeoTransform()[5])
        
        # Read values from GeoTIFF for each band
        for band_num in range(1, num_bands + 1):
            band = tiff_ds.GetRasterBand(band_num)
            value = band.ReadAsArray(px, py, 1, 1)[0][0]
            extracted_values[band_num - 1].append(value)

    df = pd.read_csv(in_wl,delimiter=" ")
    df['wl'] = round(df['wl'],2)
    df['wl'] = df['wl'].astype(str)
    wl = list(df['wl'])
    extracted_values = pd.DataFrame(np.array(extracted_values)).T
    extracted_values.columns = wl
    for col in extracted_values.columns:
        points[col] = extracted_values[col]
    points.to_file(f'{data_path}{file}')
    print("Finished:", file)

In [ ]:
data_path = "/Volumes/ChenLab-1/Fujiang/0_Seasonal_PRISMA_traits/4_Extract_training_data/2_processing_extraction/NBAR_refl/"
LAI_path = "/Volumes/ChenLab-1/Fujiang/0_Seasonal_PRISMA_traits/12_RTM_estimation_through_given_LAI/1_LAI_estimation/8_estimated_LAI_VI_masked/NBAR_refl/"

file_name = os.listdir(data_path)
file_name = [x for x in file_name if ".shp" in x and "._" not in x]

for file in file_name:
    points = gpd.read_file(f'{data_path}{file}')
    site = ("_").join(file.split("_")[2:4])
    lai_path = f"{LAI_path}{site}"
    prisma = pair_neon_prisma[file]
    
    in_tif = f"{lai_path}/{prisma}_NBAR_LAI_VI_masked.tif"
    tiff_ds = gdal.Open(in_tif)
    num_bands = tiff_ds.RasterCount
        
    extracted_values = [[] for _ in range(num_bands)]
    for index, row in points.iterrows():
        point = row.geometry
        x, y = point.x, point.y
        
        # Convert point coordinates to pixel coordinates
        px = int((x - tiff_ds.GetGeoTransform()[0]) / tiff_ds.GetGeoTransform()[1])
        py = int((y - tiff_ds.GetGeoTransform()[3]) / tiff_ds.GetGeoTransform()[5])
        
        # Read values from GeoTIFF for each band
        for band_num in range(1, num_bands + 1):
            band = tiff_ds.GetRasterBand(band_num)
            value = band.ReadAsArray(px, py, 1, 1)[0][0]
            extracted_values[band_num - 1].append(value)
    extracted_values = pd.DataFrame(np.array(extracted_values)).T
    extracted_values.columns = ["NDVI","NIRv","LAI"]
    points = pd.concat([points, extracted_values], axis = 1)
    points.to_file(f'{data_path}{file[:-4]}_add_LAI.shp')
    print("Finished:", file)

In [ ]:
shp_path = "/Volumes/ChenLab-1/Fujiang/0_Seasonal_PRISMA_traits/4_Extract_training_data/2_processing_extraction/NBAR_refl/"
out_path = "/Volumes/ChenLab-1/Fujiang/0_Seasonal_PRISMA_traits/4_Extract_training_data/3_save_to_csv/NBAR_refl/"

PRISMA_data = {'NEON_2020_D10_CPER_20200913_extracted_points_add_LAI.shp':"20200914",
               'NEON_2020_D13_MOAB_20200705_extracted_points_add_LAI.shp':"20200702",
               'NEON_2021_D03_OSBS_20210924_extracted_points_add_LAI.shp':"20210926",
               'NEON_2021_D07_MLBS_20210617_extracted_points_add_LAI.shp':"20210608",
               'NEON_2021_D13_MOAB_20210429_extracted_points_add_LAI.shp':"20210511",
               'NEON_2021_D10_CPER_20210525_extracted_points_add_LAI.shp':"20210527",
               'NEON_2021_D10_CPER_20210608_extracted_points_add_LAI.shp':"20210614",
               'NEON_2021_D14_JORN_20210826_extracted_points_add_LAI.shp':"20210817",
               'NEON_2021_D14_JORN_20210909_extracted_points_add_LAI.shp':"20210909",
               'NEON_2021_D16_WREF_20210724_extracted_points_add_LAI.shp':"20210729"}

file_name = os.listdir(shp_path)
file_name = [x for x in file_name if "_add_LAI.shp" in x and "._" not in x]
for file in file_name:
    print(file)
    data = gpd.read_file(f"{shp_path}{file}")
    data.insert(7,"site",file.split("_")[3],allow_duplicates=False)
    data.insert(8, "NEON_date", file.split("_")[4], allow_duplicates=False)
    data.insert(9, "HYP_date", PRISMA_data[file], allow_duplicates=False)
    data.insert(10, "X", data['geometry'].x, allow_duplicates=False)
    data.insert(11, "Y", data['geometry'].y, allow_duplicates=False)
    data.insert(12, "crs", data.crs.name, allow_duplicates=False)
    data.drop(columns = ["geometry"],inplace = True)
    data.to_csv(f"{out_path}{file[:-4]}.csv",index = False)

In [ ]:
csv_path = "/Volumes/ChenLab-1/Fujiang/0_Seasonal_PRISMA_traits/4_Extract_training_data/3_save_to_csv/NBAR_refl/"
out_path = "/Volumes/ChenLab-1/Fujiang/0_Seasonal_PRISMA_traits/4_Extract_training_data/5_merged_csv_data/"
file_name = os.listdir(csv_path)
file_name = [x for x in file_name if ("_add_LAI.csv" in x)&("._" not in x)]

var_start = True
for file in file_name:
    df = pd.read_csv(f"{csv_path}{file}")
    if var_start:
        results = df
        var_start = False
    else:
        results = pd.concat([results, df],axis = 0)

PFTs = {'Shrubland':"SHR", 'Open evergreen needle-leaved forest':"ENF", 'Lichens and mosses':np.nan,
        'Open mixed leaf forest (broadleaved and needle-leaved)':"MF",'Grassland': "GRA", 'Herbaceous cover cropland': "CPR",
        'Open deciduous broadleaved forest':"DBF", 'Closed evergreen needle-leaved forest':"ENF", 'Rainfed cropland':"CPR",
        'Open deciduous needle-leaved forest':"DNF", 'Irrigated cropland':"CPR",'Closed evergreen broadleaved forest':"EBF",
        'Closed deciduous broadleaved forest':"DBF", 'Sparse vegetation':np.nan,'Open evergreen broadleaved forest':"EBF",
        'Closed deciduous needle-leaved forest':"DNF", 'Impervious surfaces':np.nan}
pfts = [PFTs[x] for x in results["LULC"]]
results["PFT"] = pfts
results.dropna(subset=['PFT'],inplace = True)
results["Nitrogen"]= (results["Nitrogen"]* results["LMA"])/1000

results.dropna(subset=['LAI'], inplace = True)
results.reset_index(drop = True, inplace = True)
results.to_csv(f"{out_path}2020 and 2021 NEON extracted leaf traits and spectra data_NBAR_add_LAI.csv", index = False)